# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118806e+02     1.707169e+00
 * time: 0.08503007888793945
     1     1.067229e+01     9.326871e-01
 * time: 0.9310290813446045
     2    -1.213177e+01     1.100888e+00
 * time: 1.0427160263061523
     3    -3.432058e+01     8.178286e-01
 * time: 1.206794023513794
     4    -4.780880e+01     5.710330e-01
 * time: 1.3567969799041748
     5    -5.682517e+01     2.083896e-01
 * time: 1.4978749752044678
     6    -5.967022e+01     1.468383e-01
 * time: 1.6008031368255615
     7    -6.079532e+01     4.970938e-02
 * time: 1.7099919319152832
     8    -6.124195e+01     6.667711e-02
 * time: 1.8211450576782227
     9    -6.153005e+01     4.908919e-02
 * time: 1.9267840385437012
    10    -6.176540e+01     3.376574e-02
 * time: 2.0400121212005615
    11    -6.194908e+01     2.447534e-02
 * time: 2.146182060241699
    12    -6.202876e+01     2.020341e-02
 * time: 2.2588279247283936
    13    -6.210380e+01     1.566526e-02
 * time: 2.36413002014

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557641
    AtomicNonlocal      14.8522622
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485361 
    Xc                  -19.3336816

    total               -62.261666460361
